# Pivot and Validate Precinct Level Election Results

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

### Import the precinct level election results


#### [Open Elections - 2018 NY General Election - Precinct Level Results](data/precinct_election_results/20181106__ny__general__precinct.csv)

* Relative Path: `data/precinct_election_results/20181106__ny__general__precinct.csv`
* Link: <https://github.com/openelections/openelections-data-ny/blob/master/2018/20181106__ny__general__precinct.csv>
* Date Accessed: January 21, 2021

In [2]:
statewide_election_results_file_path = '20181106__ny__general__precinct.csv'
statewide_results_df = pd.read_csv(statewide_election_results_file_path,low_memory=False)
statewide_results_df.head(2)

,county,precinct,office,district,candidate,party,votes,election_day,absentee,machine_votes,absentee_hc,affidavit
0,Chemung,Elmira 0901,Governor,NaN,Andrew Cuomo,Dem,139,NaN,NaN,NaN,NaN,NaN
1,Chemung,Elmira 0902,Governor,NaN,Andrew Cuomo,Dem,104,NaN,NaN,NaN,NaN,NaN


The next cell aliases `statewide_results_df` as `df` because typing fewer characters allows for faster data exploration.  

In [3]:
df = statewide_results_df.copy()

### Clean `votes`

In [4]:
def remove_commas(df, col_lst):
    """
    remove commas from the string representation of numbers in the cols in col_lst
    :df: DataFrame object
    :col_lst: list of strings that are each the name of a column :df:
    """
    for col in col_lst:
        if df.dtypes[col] == "object":
            df[col] = df[col].map(lambda s: str(s).replace(",", ""))
        df = df.astype({col: "float"})
        df[col] = df[col].fillna(0)
        df = df.astype({col: "int64"})
    return df

df = remove_commas(df[df.votes.notna()], ['votes'])

/Users/baxterdemers/ny2018/venv/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


### Clean `county`

In [5]:
df.county = df.county.str.strip() + ' County'

In [6]:
df.county.nunique()

62

In [7]:
df.drop_duplicates().shape

(922315, 12)

### Clean `office`

In [8]:
def clean_office(office):
    if 'gov' in office.lower():
        return 'Governor'
    if office == 'U.S.Senate':
        return 'U.S. Senate'
    if 'U.S.House' in office:
        return 'U.S. House'
    return office.title().strip()

df.office.map(clean_office).unique()

array(['Governor', 'Comptroller', 'Attorney General', 'U.S. Senate',
       'U.S. House', 'State Senate', 'State Assembly'], dtype=object)

In [9]:
df.office = df.office.map(clean_office)

In [10]:
offices = list(df.office.unique())
print(len(offices))
print(offices)

7
['Governor', 'Comptroller', 'Attorney General', 'U.S. Senate', 'U.S. House', 'State Senate', 'State Assembly']


Ensure that all counties data for each office

In [11]:
county_to_offices = df.groupby('county').office.unique()
county_to_missing_offices = county_to_offices.apply(lambda x: set(offices).difference(x))
county_to_missing_offices[county_to_missing_offices != set()]

county
Albany County                                {State Assembly}
Allegany County                                    {Governor}
Broome County                                {State Assembly}
Chautauqua County                            {State Assembly}
Chemung County                               {State Assembly}
Clinton County                               {State Assembly}
Columbia County               {Attorney General, Comptroller}
Cortland County                              {State Assembly}
Erie County                                  {State Assembly}
Genesee County                                     {Governor}
Kings County                                 {State Assembly}
Madison County                               {State Assembly}
Montgomery County                            {State Assembly}
New York County                              {State Assembly}
Niagara County                                  {U.S. Senate}
Oneida County                                {State Assembly}
O

In [12]:
federal_offices = ['Governor', 'U.S. Senate', 'U.S. House']
county_to_missing_federal_offices = county_to_offices.apply(lambda x: set(federal_offices).difference(x))
counties_missing_federal_offices = county_to_missing_federal_offices[county_to_missing_federal_offices != set()]
print(len(counties_missing_federal_offices))
counties_missing_federal_offices

7


county
Allegany County                {Governor}
Genesee County                 {Governor}
Niagara County              {U.S. Senate}
Orange County      {Governor, U.S. House}
Saratoga County              {U.S. House}
Schuyler County                {Governor}
Suffolk County                 {Governor}
Name: office, dtype: object

Seems like Open Elections has more up to date county level files here: https://github.com/openelections/openelections-data-ny/tree/master/2018/counties

* Allegany County                {X} https://github.com/openelections/openelections-data-ny/blob/master/2018/counties/20181106__ny__general__allegany__precinct.csv
* Genesee County                 {X} https://github.com/openelections/openelections-data-ny/blob/master/2018/counties/20181106__ny__general__genesee__precinct.csv
* Niagara County              {X} https://github.com/openelections/openelections-data-ny/blob/master/2018/counties/20181106__ny__general__niagara__precinct.csv
* Orange County      {U.S. House, X} https://github.com/openelections/openelections-data-ny/blob/master/2018/counties/20181106__ny__general__orange__precinct.csv
* Saratoga County              {X} https://github.com/openelections/openelections-data-ny/blob/master/2018/counties/20181106__ny__general__saratoga__precinct.csv
* Schuyler County                {X} https://github.com/openelections/openelections-data-ny/blob/master/2018/counties/20181106__ny__general__schuyler__precinct.csv
* Suffolk County                 {X} https://github.com/openelections/openelections-data-ny/blob/master/2018/counties/20181106__ny__general__suffolk__precinct.csv 

All set with the expection of missing results for the U.S. House race in Orange County.

### Getting results for Governor in Orange County

1. Downloaded `data/parse_results/orange-county/GE18 - Gov Lt. Gov.pdf` from https://www.orangecountygov.com/DocumentCenter/View/11654/GE18---NYS-Governor--Lt-Governor
2. Used https://pdftables.com/ to convert the pdf to a csv (`data/parse_results/orange-county/GE18 - Gov Lt. Gov.csv`)
3. Made some manual tweaks to clean up the handwork of the pdf to csv converter in Numbers (like Excel for Mac OS). Saved it locally at `data/parse_results/orange-county/GE18 - Gov Lt. Gov-cleaned.csv`
4. Imported below

Since this data is already pivoted, I will incoperate it after I pivot the rest of the data.

In [13]:
df_Orange_Gov = pd.read_csv('data/parse_results/orange-county/GE18 - Gov Lt. Gov-cleaned.csv')
df_Orange_Gov

,precinct,DEM,REP,CON,GRE,WOR,IND,WEP,REF,SAM,SAM.1,SAM.2,LBT
0,Middletown - Ward 1-1,151,72,9,5,3,5,0,1,1,0,1,2
1,Middletown - Ward 1-2,76,32,4,1,1,0,1,0,0,0,0,3
2,Middletown - Ward 1-3,147,64,3,1,3,1,0,2,2,0,2,5
3,Middletown - Ward 1-4,35,20,1,1,0,2,0,0,0,0,0,1
4,Middletown - Ward 1-5,134,59,7,1,0,0,0,2,1,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,Woodbury 6,168,209,32,7,10,4,2,9,5,0,5,13
330,Woodbury 7,160,149,18,4,0,2,2,6,6,0,6,4
331,Woodbury 8,130,251,23,5,1,0,1,5,3,0,3,1
332,Woodbury 9,117,42,5,5,1,1,0,0,1,0,1,2


In [14]:
df_Orange_Gov.sum(axis=1).sum()

123446

In [15]:
df_Orange_Gov.precinct.nunique()

334

### Getting results for U.S. House in Orange County

1. Downloaded `data/parse_results/orange-county/GE18 -CD 18th.pdf` from https://www.orangecountygov.com/DocumentCenter/View/11663/GE18--CD-18th
2. Used https://pdftables.com/ to convert the pdf to a csv (`data/parse_results/orange-county/GE18 -CD 18th.csv`)
3. Made some manual tweaks to clean up the handwork of the pdf to csv converter in Numbers (like Excel for Mac OS). Saved it locally at `data/parse_results/orange-county/GE18-CD-18th-cleaned.csv`
4. Imported below

Since this data is already pivoted, I will incoperate it after I pivot the rest of the data.

In [16]:
df_Orange_HOR = pd.read_csv('data/parse_results/orange-county/GE18-CD-18th-cleaned.csv')
df_Orange_HOR

,precinct,DEM,REP,CON,WOR,IND,WEP,REF,WRITE IN,BLANK
0,Middletown - Ward 1-1,159,67,7,6,4,2,1,0,11
1,Middletown - Ward 1-2,79,34,3,1,0,0,0,0,3
2,Middletown - Ward 1-3,158,47,7,3,2,1,2,0,15
3,Middletown - Ward 1-4,36,19,0,2,1,0,0,0,3
4,Middletown - Ward 1-5,139,53,7,6,1,0,1,0,9
...,...,...,...,...,...,...,...,...,...,...
329,Woodbury 6,202,191,27,7,6,3,4,0,34
330,Woodbury 7,192,117,13,3,9,2,8,0,15
331,Woodbury 8,175,211,27,3,5,1,2,0,13
332,Woodbury 9,129,36,7,0,1,1,1,0,10


In [17]:
df_Orange_HOR.sum(axis=1).sum()

125924

In [18]:
df_Orange_HOR.precinct.nunique()

334

### Patch in Counties that are missing election results

For whatever reason, Open Elections has more complete results at the county level. Counties still missing election results for federal offices: 
* Allegany County                {Governor}
* Genesee County                 {Governor}
* Niagara County              {U.S. Senate}
* Saratoga County              {U.S. House}
* Schuyler County                {Governor}
* Suffolk County                 {Governor}

In [19]:
df_Allegany = pd.read_csv('data/county_precinct_election_results_oe/20181106__ny__general__allegany__precinct.csv')
df_Genesee = pd.read_csv('data/county_precinct_election_results_oe/20181106__ny__general__genesee__precinct.csv')
df_Niagara = pd.read_csv('data/county_precinct_election_results_oe/20181106__ny__general__niagara__precinct.csv')
df_Saratoga = pd.read_csv('data/county_precinct_election_results_oe/20181106__ny__general__saratoga__precinct.csv')
df_Schuyler = pd.read_csv('data/county_precinct_election_results_oe/20181106__ny__general__schuyler__precinct.csv')
df_Suffolk = pd.read_csv('data/county_precinct_election_results_oe/20181106__ny__general__suffolk__precinct.csv')

In [20]:
county_lst = ['Allegany', 'Genesee', 'Niagara', 'Saratoga', 'Schuyler', 'Suffolk']
county_df_lst = [df_Allegany, df_Genesee, df_Niagara, df_Saratoga, df_Schuyler,df_Suffolk]

In [21]:
for idx, county in enumerate(county_lst):
    county_df = county_df_lst[idx]
    assert  {county} == set(county_df.county)

In [22]:
df = df[~df.county.isin(county_lst)]
df = df.append(county_df_lst)

### Repeat cleaning

In [23]:
def clean_office(office):
    if 'gov' in office.lower():
        return 'Governor'
    if office == 'U.S.Senate':
        return 'U.S. Senate'
    if 'U.S.House' in office:
        return 'U.S. House'
    return office.title().strip()

df.office = df.office.map(clean_office)
df = df[df.office.isin(offices)]
df.office.unique()

array(['Governor', 'Comptroller', 'Attorney General', 'U.S. Senate',
       'U.S. House', 'State Senate', 'State Assembly'], dtype=object)

In [24]:
df.loc[df.county.isin(county_lst), 'county'] = df.loc[df.county.isin(county_lst),'county'].str.strip() + ' County'

Ensure that all counties data for each office

In [25]:
county_to_offices = df.groupby('county').office.unique()
county_to_missing_offices = county_to_offices.apply(lambda x: set(offices).difference(x))
county_to_missing_offices = county_to_missing_offices[county_to_missing_offices != set()]
print(len(county_to_missing_offices))
county_to_missing_offices

21


county
Albany County                                {State Assembly}
Broome County                                {State Assembly}
Chautauqua County                            {State Assembly}
Chemung County                               {State Assembly}
Clinton County                               {State Assembly}
Columbia County               {Attorney General, Comptroller}
Cortland County                              {State Assembly}
Erie County                                  {State Assembly}
Kings County                                 {State Assembly}
Madison County                               {State Assembly}
Montgomery County                            {State Assembly}
New York County                              {State Assembly}
Oneida County                                {State Assembly}
Orange County          {State Assembly, Governor, U.S. House}
Queens County                                   {Comptroller}
Richmond County                              {State Assembly}
S

In [26]:
federal_offices = ['Governor', 'U.S. Senate', 'U.S. House']
county_to_missing_federal_offices = county_to_offices.apply(lambda x: set(federal_offices).difference(x))
counties_missing_federal_offices = county_to_missing_federal_offices[county_to_missing_federal_offices != set()]
print(len(counties_missing_federal_offices))
counties_missing_federal_offices

1


county
Orange County    {Governor, U.S. House}
Name: office, dtype: object

Exclude non-federal offices because so much source data is missing

In [27]:
df = df[df.office.isin(federal_offices)]

In [28]:
df_cleaned_offices = df.copy()

In [29]:
df_cleaned_offices[(df_cleaned_offices.county=='Genesee County') & df_cleaned_offices.party.isna()].candidate.unique()
df.loc[(df.county=='Genesee County') & (df.candidate=='Gillibrand')] = ''

In [30]:
df.loc[(df.county=='Genesee County') & (df.candidate=='Gillibrand')]

,county,precinct,office,district,candidate,party,votes,election_day,absentee,machine_votes,absentee_hc,affidavit,Unnamed: 7,Unnamed: 8


In [31]:
df_cleaned_offices[(df_cleaned_offices.county=='Genesee County') & df_cleaned_offices.party.isna()]

,county,precinct,office,district,candidate,party,votes,election_day,absentee,machine_votes,absentee_hc,affidavit,Unnamed: 7,Unnamed: 8
497784,Genesee County,Alabama 1,U.S. Senate,NaN,Gillibrand,NaN,105,NaN,NaN,NaN,NaN,NaN,NaN,NaN
497785,Genesee County,Alabama 1,U.S. Senate,NaN,CC Farley,NaN,185,NaN,NaN,NaN,NaN,NaN,NaN,NaN
497786,Genesee County,Alabama 1,U.S. Senate,NaN,CC Farley,NaN,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN
497787,Genesee County,Alabama 1,U.S. Senate,NaN,Gillibrand,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
497788,Genesee County,Alabama 1,U.S. Senate,NaN,Gillibrand,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2792,Genesee County,Ward 6-2,U.S. House,27,Collins,NaN,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2793,Genesee County,Ward 6-2,U.S. House,27,McMurray,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2794,Genesee County,Ward 6-2,U.S. House,27,Collins,NaN,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2795,Genesee County,Ward 6-2,U.S. House,27,McMurray,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Clean `party`

The oficial parties in New York are [defined as](https://www.elections.ny.gov/CFCommittees.html)

- Democratic
- Republican
- Conservative
- Working Families
- Green
- Libertarian
- Independence
- SAM

Since the official results also included `WEP (Women's Equality Party)` and `REF (Reform Party of New York State)` I will include those as well.

In [32]:
df.party.nunique()

74

In [33]:
df.groupby('county').party.nunique().sort_values()

county
Genesee County        0
                      1
Orange County         9
Monroe County        10
Nassau County        10
                     ..
Clinton County       17
Kings County         17
Wayne County         17
Rensselaer County    20
Rockland County      23
Name: party, Length: 63, dtype: int64

Genesee is missing party labels for candidates. Need to add them in before pivoting.

In [34]:
official_party_to_abbreviation = {
    "Democratic":"DEM",
    "Republican":"REP",
    "Conservative":"CON",
    "Working Families":"WOR",
    "Green":"GRE",
    "Libertarian":"LBT",
    "Independence":"IND",
    "SAM": "SAM",
    "Women's Equality Party":"WEP",
    "Reform Party":"REF",
}

parties = set(official_party_to_abbreviation.values())
misc = {'BLANK, VOID OR SCATTERED', 'WRITEIN', 'OVERVOTE', 'UNDERVOTE', 'UNQUALIFIED WRITE-INS'}

In [35]:
party_lookup_table = {
    'DEMOCRATIC':'DEM',
    '0':'BLANK, VOID OR SCATTERED',
    '1635':'BLANK, VOID OR SCATTERED',
    '4328':'BLANK, VOID OR SCATTERED',
    'BLANK & VOID':'BLANK, VOID OR SCATTERED',
    'BLANKS':'BLANK, VOID OR SCATTERED',
    'BLANKS & VOID':'BLANK, VOID OR SCATTERED',
    'IND': 'IND',
    'REP': 'REP',
 'Write': 'Write',
 'DEM': 'DEM',
 'CON': 'CON',
 'SAM': 'SAM',
     'GRE': 'GRE',
 'WOR': 'WOR',
 'REF': 'REF',
 'WEP': 'WEP',
 'LBT': 'LBT',
 'WRI':'WRITEIN',
 'WRITE IN':'WRITEIN',
 'WRITE INS':'WRITEIN',
 'WRITE-IN':'WRITEIN',
 'WRITE-INS':'WRITEIN',
 'WRITEIN':'WRITEIN',
 'WRITEINS':'WRITEIN',
     'VOID':'BLANK, VOID OR SCATTERED',
 'VOIDS':'BLANK, VOID OR SCATTERED',
 'VOIDS/BLANKS':'BLANK, VOID OR SCATTERED',
    'GREEN':'GRE',
 'GRN':'GRE',
     'BLANK':'BLANK, VOID OR SCATTERED',
 'BLANK VOTES':'BLANK, VOID OR SCATTERED',
 'BLANK/VOID':'BLANK, VOID OR SCATTERED',
     'LIB':'LBT',
 'LIBERTARIAN':'LBT',
    'SCATTER':'BLANK, VOID OR SCATTERED',
 'SCATTERED':'BLANK, VOID OR SCATTERED',
 'SCATTERING':'BLANK, VOID OR SCATTERED',
 'SCATTERINGS':'BLANK, VOID OR SCATTERED',
'CONSERVATIVE':'CON',
         'OVER':'OVERVOTE',
 'OVER VOTES':'OVERVOTE',
 'OVERVOTE':'OVERVOTE',
 'OVERVOTES':'OVERVOTE',
 'UNDER':'UNDERVOTE',
 'UNDER VOTES':'UNDERVOTE',
 'UNDER/OVER':'UNDERVOTE',
 'UNDER/OVER VOTES':'UNDERVOTE',
 'UNDERVOTE':'UNDERVOTE',
 'UNDERVOTES':'UNDERVOTE',
    'REPUBLICAN':'REP',
    'REFORM':'REF',
   'REFORM PARTY':'REF',
    'INDEPENDENCE':'IND',
    'WFP':'WOR',
    'WF':'WOR',
    "WOMEN'S EQUALITY PARTY":'WEP',
    "WOMEN'S EQUALITY":"WEP",
    # ballotpedia lookups
    'GLOGOWSKI':'LBT',
    "CHRISTOPHER B. GARVEY":'LBT',
    'SEAMANS':'DEM',
    'DINAPOLI':'DEM',
    'GILLIBRAND':'DEM',
    'JAMES': 'DEM',
    'COLLINS':'REP',
    'TRICHTER':'REP',
    'CC FARLEY':'REP',
    'WOFFORD':'REP',
    'RANZENHOFER':'REP',
    'HAWLEY':'REP',
    'MCMURRAY':'DEM',
}

def clean_party(party):
    if (type(party) != str):
        return 'nS'
    cleaned_party = party.strip().upper()
    looked_up_party = party_lookup_table.get(cleaned_party, cleaned_party)
    return official_party_to_abbreviation.get(looked_up_party.title(), looked_up_party)

In [36]:
df = df.loc[~(df.party.str.upper().isin([
        'BALLOTS CAST',
        'TOTAL',
        'TOTAL BALLOTS',
        'TOTAL VOTES',
        'REGISTERED VOTERS',
        'FEDERAL',
        'FEDERAL BALLOTS',
        'FEDERAL VOTES',
        'STATE BALLOTS',
        'STATE VOTES',
        'REGISTERED',
        'PUBLIC COUNTER',]))
        ]

In [37]:
df['party'] = df.party.map(clean_party)

In [38]:
df[df.county =='Genesee County'].office.unique()

array(['U.S. Senate', 'U.S. House', 'Governor'], dtype=object)

In [39]:
df['party'] = df.party.map(clean_party)
df = df[df.party.isin(parties)]
value_counts = df.groupby('party').votes.sum()
print(value_counts[value_counts > 1000].shape)
value_counts[value_counts > 1000].sort_values().tail(60)

(10,)


party
SAM       62454
LBT      103878
REF      108673
WEP      131414
GRE      139193
IND      234210
WOR      563466
CON      836638
REP     6173122
DEM    13077951
Name: votes, dtype: int64

In [40]:
df = df[df.party.isin(parties)]
value_counts = df.groupby('party').votes.sum()
print(value_counts[value_counts > 1000].shape)
value_counts[value_counts > 1000].sort_values()

(10,)


party
SAM       62454
LBT      103878
REF      108673
WEP      131414
GRE      139193
IND      234210
WOR      563466
CON      836638
REP     6173122
DEM    13077951
Name: votes, dtype: int64

In [41]:
parties

{'CON', 'DEM', 'GRE', 'IND', 'LBT', 'REF', 'REP', 'SAM', 'WEP', 'WOR'}

In [42]:
df[df.county =='Genesee County'].office.unique()

array([], dtype=object)

In [43]:
df.office.unique()

array(['Governor', 'U.S. Senate', 'U.S. House'], dtype=object)

In [44]:
office_to_abbreviation = {
    'Governor':'Gov', 
    'Comptroller':'Compt', 
    'Attorney General':'AG',
    'U.S. Senate':'Sen',
    'U.S. House':'HOR',
    'State Senate':'StSen', 
    'State Assembly':'StAssy'
}

In [45]:
df.office = df.office.map(office_to_abbreviation)

In [46]:
df.head()

,county,precinct,office,district,candidate,party,votes,election_day,absentee,machine_votes,absentee_hc,affidavit,Unnamed: 7,Unnamed: 8
0,Chemung County,Elmira 0901,Gov,NaN,Andrew Cuomo,DEM,139,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Chemung County,Elmira 0902,Gov,NaN,Andrew Cuomo,DEM,104,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Chemung County,Elmira 0903,Gov,NaN,Andrew Cuomo,DEM,84,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Chemung County,Elmira 0904,Gov,NaN,Andrew Cuomo,DEM,64,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Chemung County,Elmira 0905,Gov,NaN,Andrew Cuomo,DEM,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
# remove empty columns and drop dups + `total` precs
df = df[['county', 'precinct', 'office', 'district', 'candidate', 'party', 'votes']]
df = df.drop_duplicates()
precincts_to_remove_lower = ['total', 'county totals', 'tompkins ny', 'cortland ny', 'city of buffalo total']
df = df[~df.precinct.str.lower().isin(precincts_to_remove_lower)]

In [48]:
piv = pd.pivot_table(df, index = ['county', 'precinct'], columns = ['office','party'], values = ['votes'], aggfunc = np.sum)

In [49]:
piv.columns = [office + '_' + party for _,office,party in piv.columns]
piv.head()

Gov_CON  Gov_DEM  Gov_GRE  Gov_IND  Gov_LBT  \
county        precinct                                                          
Albany County 0001 ALBANY W1 ED1      1.0     35.0      3.0      0.0      1.0   
              0002 ALBANY W1 ED2     11.0    148.0     15.0      1.0      3.0   
              0003 ALBANY W1 ED3     11.0    212.0      4.0      1.0      5.0   
              0004 ALBANY W1 ED4      5.0    220.0      4.0      4.0      4.0   
              0005 ALBANY W1 ED5      0.0      2.0      1.0      0.0      0.0   

                                  Gov_REF  Gov_REP  Gov_SAM  Gov_WEP  Gov_WOR  \
county        precinct                                                          
Albany County 0001 ALBANY W1 ED1      0.0      9.0      2.0      0.0      3.0   
              0002 ALBANY W1 ED2      2.0     32.0      9.0      1.0     11.0   
              0003 ALBANY W1 ED3      0.0     55.0      5.0      3.0     10.0   
              0004 ALBANY W1 ED4      3.0     30.0      7.0      1.0     12.0   
              0005 ALBANY W1 ED5      0.0      0.0      0.0      0.0      0.0   

                                  ...  HOR_REP  HOR_WEP  HOR_WOR  Sen_CON  \
county        precinct            ...                                       
Albany County 0001 ALBANY W1 ED1  ...      7.0      1.0      5.0      1.0   
              0002 ALBANY W1 ED2  ...     20.0      6.0     21.0      5.0   
              0003 ALBANY W1 ED3  ...     41.0      1.0     11.0     11.0   
              0004 ALBANY W1 ED4  ...     18.0      5.0     18.0      6.0   
              0005 ALBANY W1 ED5  ...      0.0      0.0      0.0      0.0   

                                  Sen_DEM  Sen_IND  Sen_REF  Sen_REP  Sen_WEP  \
county        precinct                                                          
Albany County 0001 ALBANY W1 ED1     40.0      0.0      0.0      8.0      0.0   
              0002 ALBANY W1 ED2    176.0      4.0      1.0     23.0      4.0   
              0003 ALBANY W1 ED3    228.0      5.0      0.0     40.0      5.0   
              0004 ALBANY W1 ED4    241.0      5.0      3.0     15.0      4.0   
              0005 ALBANY W1 ED5      4.0      0.0      0.0      0.0      0.0   

                                  Sen_WOR  
county        precinct                     
Albany County 0001 ALBANY W1 ED1      6.0  
              0002 ALBANY W1 ED2     22.0  
              0003 ALBANY W1 ED3     10.0  
              0004 ALBANY W1 ED4     15.0  
              0005 ALBANY W1 ED5      0.0  

[5 rows x 25 columns]

In [50]:
piv.groupby('county').sum()

,Gov_CON,Gov_DEM,Gov_GRE,Gov_IND,Gov_LBT,Gov_REF,Gov_REP,Gov_SAM,Gov_WEP,Gov_WOR,...,HOR_REP,HOR_WEP,HOR_WOR,Sen_CON,Sen_DEM,Sen_IND,Sen_REF,Sen_REP,Sen_WEP,Sen_WOR
county,,,,,,,,,,,,,,,,,,,,,
Albany County,6725.0,54909.0,3543.0,1408.0,1867.0,835.0,38357.0,3007.0,886.0,2489.0,...,32326.0,1640.0,4781.0,6034.0,70182.0,2655.0,597.0,29825.0,1395.0,3883.0
Allegany County,833.0,12488.0,183.0,106.0,963.0,65.0,0.0,117.0,46.0,80.0,...,2252.0,0.0,0.0,892.0,4844.0,222.0,64.0,8305.0,99.0,295.0
Bronx County,2407.0,253712.0,2828.0,2143.0,1074.0,271.0,20401.0,593.0,503.0,4198.0,...,14712.0,1204.0,8339.0,2590.0,249485.0,2369.0,331.0,20112.0,698.0,5171.0
Broome County,3244.0,29065.0,1320.0,931.0,2000.0,474.0,32865.0,802.0,379.0,1144.0,...,29452.0,469.0,1774.0,2910.0,37053.0,1588.0,331.0,27656.0,597.0,1907.0
Cattaraugus County,1629.0,6545.0,307.0,265.0,1060.0,159.0,13240.0,192.0,78.0,146.0,...,13173.0,239.0,444.0,1556.0,9199.0,556.0,114.0,11586.0,179.0,497.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Washington County,1395.0,5964.0,453.0,198.0,571.0,154.0,11330.0,199.0,82.0,167.0,...,10263.0,182.0,437.0,1255.0,9248.0,499.0,89.0,8596.0,168.0,550.0
Wayne County,2765.0,8655.0,435.0,255.0,1648.0,262.0,16547.0,461.0,114.0,217.0,...,15918.0,338.0,625.0,2712.0,12374.0,0.0,218.0,14592.0,267.0,660.0
Westchester County,10295.0,212833.0,3738.0,4692.0,1892.0,1100.0,90785.0,2576.0,1865.0,3295.0,...,10079.0,3753.0,9666.0,10789.0,221750.0,4190.0,967.0,85222.0,2277.0,5956.0


### Add in Gov results for Orange County

Need to add a multi index to `df_Orange_Gov` to allow for indexed assignment with `piv` (which has a multi index) 

In [51]:
df_Orange_Gov = df_Orange_Gov.set_index('precinct')
df_Orange_Gov['county'] = 'Orange County'
df_Orange_Gov.set_index('county', append=True, inplace=True)
df_Orange_Gov = df_Orange_Gov.reorder_levels(['county', 'precinct'])
df_Orange_Gov.head()

DEM  REP  CON  GRE  WOR  IND  WEP  REF  \
county        precinct                                                        
Orange County Middletown - Ward 1-1  151   72    9    5    3    5    0    1   
              Middletown - Ward 1-2   76   32    4    1    1    0    1    0   
              Middletown - Ward 1-3  147   64    3    1    3    1    0    2   
              Middletown - Ward 1-4   35   20    1    1    0    2    0    0   
              Middletown - Ward 1-5  134   59    7    1    0    0    0    2   

                                     SAM  SAM.1  SAM.2  LBT  
county        precinct                                       
Orange County Middletown - Ward 1-1    1      0      1    2  
              Middletown - Ward 1-2    0      0      0    3  
              Middletown - Ward 1-3    2      0      2    5  
              Middletown - Ward 1-4    0      0      0    1  
              Middletown - Ward 1-5    1      0      1    2

Along the same lines, sorting the indices is needed so they line up.

In [52]:
piv = piv.sort_index()
df_Orange_Gov = df_Orange_Gov.sort_index()
list(piv.loc['Orange County'].index) == list(df_Orange_Gov.loc['Orange County','DEM'].index)

True

Now I can make the substitutions

In [53]:
[x for x in piv.columns if 'Gov' in x]

['Gov_CON',
 'Gov_DEM',
 'Gov_GRE',
 'Gov_IND',
 'Gov_LBT',
 'Gov_REF',
 'Gov_REP',
 'Gov_SAM',
 'Gov_WEP',
 'Gov_WOR']

In [54]:
piv.loc['Orange County', 'Gov_CON'] = df_Orange_Gov['CON']
piv.loc['Orange County', 'Gov_DEM'] = df_Orange_Gov['DEM']
piv.loc['Orange County', 'Gov_GRE'] = 0 # not present in the results for orange county
piv.loc['Orange County', 'Gov_IND'] = df_Orange_Gov['IND']
piv.loc['Orange County', 'Gov_REF'] = df_Orange_Gov['REF']
piv.loc['Orange County', 'Gov_REP'] = df_Orange_Gov['REP']
piv.loc['Orange County', 'Gov_WEP'] = df_Orange_Gov['WEP']
piv.loc['Orange County', 'Gov_WOR'] = df_Orange_Gov['WOR']

In [55]:
piv.loc['Orange County'].head(2)

,Gov_CON,Gov_DEM,Gov_GRE,Gov_IND,Gov_LBT,Gov_REF,Gov_REP,Gov_SAM,Gov_WEP,Gov_WOR,...,HOR_REP,HOR_WEP,HOR_WOR,Sen_CON,Sen_DEM,Sen_IND,Sen_REF,Sen_REP,Sen_WEP,Sen_WOR
precinct,,,,,,,,,,,,,,,,,,,,,
Blooming Grove 1-1,9.0,99.0,0.0,0.0,NaN,5.0,82.0,NaN,0.0,1.0,...,NaN,NaN,NaN,11.0,116.0,1.0,3.0,68.0,0.0,2.0
Blooming Grove 1-2,25.0,225.0,0.0,12.0,NaN,8.0,200.0,NaN,2.0,3.0,...,NaN,NaN,NaN,20.0,260.0,10.0,6.0,175.0,5.0,10.0


### Add in HOR results for Orange County

Need to add a multi index to `df_Orange_HOR` to allow for indexed assignment with `piv` (which has a multi index) 

In [56]:
df_Orange_HOR = df_Orange_HOR.set_index('precinct')
df_Orange_HOR['county'] = 'Orange County'
df_Orange_HOR.set_index('county', append=True, inplace=True)
df_Orange_HOR = df_Orange_HOR.reorder_levels(['county', 'precinct'])
df_Orange_HOR.head()

DEM  REP  CON  WOR  IND  WEP  REF  \
county        precinct                                                   
Orange County Middletown - Ward 1-1  159   67    7    6    4    2    1   
              Middletown - Ward 1-2   79   34    3    1    0    0    0   
              Middletown - Ward 1-3  158   47    7    3    2    1    2   
              Middletown - Ward 1-4   36   19    0    2    1    0    0   
              Middletown - Ward 1-5  139   53    7    6    1    0    1   

                                     WRITE IN  BLANK  
county        precinct                                
Orange County Middletown - Ward 1-1         0     11  
              Middletown - Ward 1-2         0      3  
              Middletown - Ward 1-3         0     15  
              Middletown - Ward 1-4         0      3  
              Middletown - Ward 1-5         0      9

Along the same lines, sorting the indices is needed so they line up.

In [57]:
piv = piv.sort_index()
df_Orange_HOR = df_Orange_HOR.sort_index()
list(piv.loc['Orange County'].index) == list(df_Orange_HOR.loc['Orange County','DEM'].index)

True

Now I can make the substitutions

In [58]:
[x for x in piv.columns if 'HOR' in x]

['HOR_CON',
 'HOR_DEM',
 'HOR_GRE',
 'HOR_IND',
 'HOR_REF',
 'HOR_REP',
 'HOR_WEP',
 'HOR_WOR']

In [59]:
piv.loc['Orange County', 'HOR_CON'] = df_Orange_HOR['CON']
piv.loc['Orange County', 'HOR_DEM'] = df_Orange_HOR['DEM']
piv.loc['Orange County', 'HOR_GRE'] = 0 # not present in the results for orange county
piv.loc['Orange County', 'HOR_IND'] = df_Orange_HOR['IND']
piv.loc['Orange County', 'HOR_REF'] = df_Orange_HOR['REF']
piv.loc['Orange County', 'HOR_REP'] = df_Orange_HOR['REP']
piv.loc['Orange County', 'HOR_WEP'] = df_Orange_HOR['WEP']
piv.loc['Orange County', 'HOR_WOR'] = df_Orange_HOR['WOR']

In [60]:
piv.loc['Orange County'].head(2)

,Gov_CON,Gov_DEM,Gov_GRE,Gov_IND,Gov_LBT,Gov_REF,Gov_REP,Gov_SAM,Gov_WEP,Gov_WOR,...,HOR_REP,HOR_WEP,HOR_WOR,Sen_CON,Sen_DEM,Sen_IND,Sen_REF,Sen_REP,Sen_WEP,Sen_WOR
precinct,,,,,,,,,,,,,,,,,,,,,
Blooming Grove 1-1,9.0,99.0,0.0,0.0,NaN,5.0,82.0,NaN,0.0,1.0,...,64.0,0.0,1.0,11.0,116.0,1.0,3.0,68.0,0.0,2.0
Blooming Grove 1-2,25.0,225.0,0.0,12.0,NaN,8.0,200.0,NaN,2.0,3.0,...,183.0,0.0,8.0,20.0,260.0,10.0,6.0,175.0,5.0,10.0


### Patch in Counties that didn't match the official results

Open Elections source data didn't match the official results for the following so I went to county websites to get their data directly:

#### Jefferson County
1. Download precinct level results from the Jefferson County [website](https://co.jefferson.ny.us/departments/Elections/election-results)
2. Made some manual tweaks to clean up the handwork of the pdf to csv converter in Numbers (like Excel for Mac OS). Saved it locally at `data/parse_results/jefferson-county`
3. Imported below

In [61]:
df_Jefferson_Gov = pd.read_csv('data/parse_results/jefferson-county/Gov.csv')
df_Jefferson_Gov = df_Jefferson_Gov.set_index('precinct')
df_Jefferson_Gov.columns = ['Gov_' + col for col in df_Jefferson_Gov.columns]
df_Jefferson_Gov.head(2)

,Gov_DEM,Gov_REP,Gov_CON,Gov_GRE,Gov_WOR,Gov_IND,Gov_WEP,Gov_REF,Gov_SAM,Gov_LBT
precinct,,,,,,,,,,
12/1,327,353,52,9,2,10,1,4,27,11
12/2,256,283,31,9,5,9,2,1,19,21


In [62]:
df_Jefferson_HOR = pd.read_csv('data/parse_results/jefferson-county/HOR.csv')
df_Jefferson_HOR = df_Jefferson_HOR.set_index('precinct')
df_Jefferson_HOR.columns = ['HOR_' + col for col in df_Jefferson_HOR.columns]
df_Jefferson_HOR.head(2)

,HOR_DEM,HOR_REP,HOR_CON,HOR_GRE,HOR_WOR,HOR_IND,HOR_WEP,HOR_REF
precinct,,,,,,,,
12/1,372,361,51,13,6,10,7,1
12/2,306,283,29,17,9,12,3,2


In [63]:
df_Jefferson_Sen = pd.read_csv('data/parse_results/jefferson-county/Sen.csv')
df_Jefferson_Sen = df_Jefferson_Sen.set_index('precinct')
df_Jefferson_Sen.columns = ['Sen_' + col for col in df_Jefferson_Sen.columns]
df_Jefferson_Sen.head(2)

,Sen_DEM,Sen_REP,Sen_CON,Sen_WOR,Sen_IND,Sen_WEP,Sen_REF
precinct,,,,,,,
12/1,454,274,35,14,21,6,1
12/2,353,230,24,13,20,3,5


In [64]:
df_Jefferson = df_Jefferson_HOR.join([df_Jefferson_Sen, df_Jefferson_Gov])
df_Jefferson['county'] = 'Jefferson County'
df_Jefferson.set_index('county', append=True, inplace=True)
df_Jefferson = df_Jefferson.reorder_levels(['county', 'precinct'])
df_Jefferson.head(2)

HOR_DEM  HOR_REP  HOR_CON  HOR_GRE  HOR_WOR  \
county           precinct                                                
Jefferson County 12/1          372      361       51       13        6   
                 12/2          306      283       29       17        9   

                           HOR_IND  HOR_WEP  HOR_REF  Sen_DEM  Sen_REP  ...  \
county           precinct                                               ...   
Jefferson County 12/1           10        7        1      454      274  ...   
                 12/2           12        3        2      353      230  ...   

                           Gov_DEM  Gov_REP  Gov_CON  Gov_GRE  Gov_WOR  \
county           precinct                                                
Jefferson County 12/1          327      353       52        9        2   
                 12/2          256      283       31        9        5   

                           Gov_IND  Gov_WEP  Gov_REF  Gov_SAM  Gov_LBT  
county           precinct                                               
Jefferson County 12/1           10        1        4       27       11  
                 12/2            9        2        1       19       21  

[2 rows x 25 columns]

In [65]:
# Add it into the rest of the results for the state
piv.drop([('Jefferson County', precinct) for precinct in piv.loc['Jefferson County'].index], inplace=True)
piv = pd.concat([df_Jefferson, piv])

#### Madison County
1. Downloaded `data/parse_results/madison-county/United States Senator.pdf` from https://www.madisoncounty.ny.gov/DocumentCenter/View/7554/United-States-Senator
2. Used https://pdftables.com/ to convert the pdf to a csv (`data/parse_results/madison-county/United States Senator.csv`)
3. Made some manual tweaks to clean up the handwork of the pdf to csv converter in Numbers (like Excel for Mac OS). Saved it locally at `data/parse_results/madison-county/United States Senator-cleaned.csv`
4. Imported below

In [66]:
df_Madison_Sen = pd.read_csv('data/parse_results/madison-county/United States Senator-cleaned.csv')
df_Madison_Sen.head()

,precinct,DEM,REP,CON,WOR,IND,WEP,REF
0,City of Oneida 1,373,346,47,17,14,9,5
1,City of Oneida 2,322,295,36,9,22,8,7
2,City of Oneida 3,389,283,44,17,20,5,6
3,City of Oneida 4,127,116,20,4,7,3,1
4,City of Oneida 5,178,173,20,7,9,6,12


In [67]:
df_Madison_Sen = df_Madison_Sen.set_index('precinct')
df_Madison_Sen['county'] = 'Madison County'
df_Madison_Sen.set_index('county', append=True, inplace=True)
df_Madison_Sen = df_Madison_Sen.reorder_levels(['county', 'precinct'])
df_Madison_Sen.head()

DEM  REP  CON  WOR  IND  WEP  REF
county         precinct                                           
Madison County City of Oneida 1  373  346   47   17   14    9    5
               City of Oneida 2  322  295   36    9   22    8    7
               City of Oneida 3  389  283   44   17   20    5    6
               City of Oneida 4  127  116   20    4    7    3    1
               City of Oneida 5  178  173   20    7    9    6   12

In [68]:
piv = piv.sort_index()
df_Madison_Sen = df_Madison_Sen.sort_index()
list(piv.loc['Madison County'].index) == list(df_Madison_Sen.loc['Madison County','DEM'].index)

True

Now I can make the substitutions

In [69]:
[x for x in piv.columns if 'Sen' in x]

['Sen_DEM', 'Sen_REP', 'Sen_CON', 'Sen_WOR', 'Sen_IND', 'Sen_WEP', 'Sen_REF']

In [70]:
piv.loc['Madison County', 'Sen_CON'] = df_Madison_Sen['CON']
piv.loc['Madison County', 'Sen_DEM'] = df_Madison_Sen['DEM']
piv.loc['Madison County', 'Sen_IND'] = df_Madison_Sen['IND']
piv.loc['Madison County', 'Sen_REF'] = df_Madison_Sen['REF']
piv.loc['Madison County', 'Sen_REP'] = df_Madison_Sen['REP']
piv.loc['Madison County', 'Sen_WEP'] = df_Madison_Sen['WEP']
piv.loc['Madison County', 'Sen_WOR'] = df_Madison_Sen['WOR']

In [71]:
piv.loc['Madison County'].head(2)

,HOR_DEM,HOR_REP,HOR_CON,HOR_GRE,HOR_WOR,HOR_IND,HOR_WEP,HOR_REF,Sen_DEM,Sen_REP,...,Gov_DEM,Gov_REP,Gov_CON,Gov_GRE,Gov_WOR,Gov_IND,Gov_WEP,Gov_REF,Gov_SAM,Gov_LBT
precinct,,,,,,,,,,,,,,,,,,,,,
City of Oneida 1,370.0,381.0,42.0,NaN,11.0,17.0,7.0,2.0,373.0,346.0,...,262.0,420.0,47.0,18.0,8.0,6.0,6.0,6.0,16.0,30.0
City of Oneida 2,334.0,295.0,33.0,NaN,12.0,28.0,2.0,4.0,322.0,295.0,...,233.0,347.0,43.0,6.0,4.0,13.0,2.0,4.0,38.0,22.0


#### Lewis County

https://www.lewiscounty.org/media/Board%20of%20Elections/Election%20Results/2018/GE18%20Election%20Results

Not sure how Lewis County counts results... 

Does Abs/Aff need to get added to the data?

If yes: then need to parse the pdf and manually add in the abs/aff column.
If no: official county results are wrong and need to be corrected, but can use the OE data as is

## TODO
* [_] Validate Results for Gov
* [_] Pull more county level official results and do sen + HOR?

## Validation

Compare the precinct level election results to official county level election results released by New York's Secretary of State.

Official County Level Election Results Source: https://www.elections.ny.gov/2018ElectionResults.html

### Clean up the pivoted data

In [72]:
piv = remove_commas(piv.fillna(0), piv.columns)

### Compare Senate Results

In [73]:
office_prefix = 'Sen'

In [74]:
#import official county level results for the gubernatorial race
official_county_level_election_results_file_path = 'data/official_county_level_election_results/2018USSenate.csv'
official_county_level_election_results_df_sen = pd.read_csv(official_county_level_election_results_file_path,low_memory=False)
official_county_level_election_results_df_sen = official_county_level_election_results_df_sen.set_index('County')
official_county_level_election_results_df_sen.index = official_county_level_election_results_df_sen.index.str.strip()
official_county_level_election_results_df_sen = remove_commas(official_county_level_election_results_df_sen, official_county_level_election_results_df_sen.columns)
official_county_level_election_results_df_sen.head(2)

,Kirsten E. Gillibrand (DEM),Chele Chiavacci Farley (REP),Chele Chiavacci Farley (CON),Kirsten E. Gillibrand (WOR),Kirsten E. Gillibrand (IND),Kirsten E. Gillibrand (WEP),Chele Chiavacci Farley (REF),Blank,Void,Scattering,Total Votes by County,Unnamed: 12
County,,,,,,,,,,,,
Albany County,70182,29825,6034,3883,2655,1395,597,2545,56,124,117296,0
Allegany County,4844,8305,892,295,222,99,64,626,3,3,15353,0


In [75]:
o_sen = official_county_level_election_results_df_sen[[x for x in official_county_level_election_results_df_sen.columns if '(' in x]]
o_sen.columns = [x.split('(')[1][:3] for x in o_sen.columns]
o_sen.head()

,DEM,REP,CON,WOR,IND,WEP,REF
County,,,,,,,
Albany County,70182,29825,6034,3883,2655,1395,597
Allegany County,4844,8305,892,295,222,99,64
Broome County,37053,27656,2910,1907,1588,597,331
Cattaraugus County,9199,11586,1556,497,556,179,114
Cayuga County,12339,11318,1772,591,666,227,142


In [76]:
res_sen = piv.groupby('county').sum()[[x for x in piv.columns if x.startswith(office_prefix)]]
res_sen.columns = [x.split('_')[1] for x in res_sen.columns]
res_sen.head()

,DEM,REP,CON,WOR,IND,WEP,REF
county,,,,,,,
Albany County,70182,29825,6034,3883,2655,1395,597
Allegany County,4844,8305,892,295,222,99,64
Bronx County,249485,20112,2590,5171,2369,698,331
Broome County,37053,27656,2910,1907,1588,597,331
Cattaraugus County,9199,11586,1556,497,556,179,114


#### Total votes cast for the race

In [96]:
percent_diff = res_sen.sum(axis=1)/o_sen.sum(axis=1)
acceptable_percent_diff = .02
lower_bound = 1 - acceptable_percent_diff
upper_bound = 1 + acceptable_percent_diff
print("Accepting ratios between {} and {}".format(lower_bound, upper_bound))
print("The outliers are: ")
outliers = percent_diff[(percent_diff < lower_bound) | (percent_diff > upper_bound)]
outliers

Accepting ratios between 0.98 and 1.02
The outliers are: 


Lewis County       0.926589
Onondaga County    0.967235
Wayne County       0.979970
Wyoming County     1.049214
dtype: float64

In [97]:
total_votes_diff = res_sen.sum(axis=1) - o_sen.sum(axis=1)
non_zero_votes_diff = total_votes_diff[total_votes_diff.index.isin(outliers.index)]
non_zero_votes_diff

Lewis County       -648.0
Onondaga County   -5760.0
Wayne County       -630.0
Wyoming County      648.0
dtype: float64

In [98]:
o_sen[o_sen.index.isin(outliers.index)]

,DEM,REP,CON,WOR,IND,WEP,REF
County,,,,,,,
Lewis County,3260,4593,490,197,193,57,37
Onondaga County,98804,55240,10455,3833,4855,1709,904
Wayne County,12374,14592,2712,660,630,267,218
Wyoming County,3766,7553,1215,280,214,70,69


In [99]:
res_sen[res_sen.index.isin(outliers.index)]

,DEM,REP,CON,WOR,IND,WEP,REF
county,,,,,,,
Lewis County,2981,4298,449,187,181,49,34
Onondaga County,98804,51013,9695,3600,4486,1601,841
Wayne County,12374,14592,2712,660,0,267,218
Wyoming County,4043,7852,1252,289,224,81,74


### Compare Gubernatorial results

In [81]:
#import official county level results for the gubernatorial race
official_county_level_election_results_file_path_gov = 'data/official_county_level_election_results/2018Governor.csv'
official_county_level_election_results_df_gov = pd.read_csv(official_county_level_election_results_file_path_gov,low_memory=False)
official_county_level_election_results_df_gov = official_county_level_election_results_df_gov.set_index('County')
official_county_level_election_results_df_gov.index = official_county_level_election_results_df_gov.index.str.strip()
official_county_level_election_results_df_gov = remove_commas(official_county_level_election_results_df_gov, official_county_level_election_results_df_gov.columns[:-4])
official_county_level_election_results_df_gov.head(2)

,Andrew M. Cuomo and Kathy C. Hochul (DEM),Marc Molinaro and Julie Killian (REP),Marc Molinaro and Julie Killian (CON),Howie Hawkins and Jia Lee (GRE),Andrew M. Cuomo and Kathy C. Hochul (WOR),Andrew M. Cuomo and Kathy C. Hochul (IND),Andrew M. Cuomo and Kathy C. Hochul (WEP),Marc Molinaro and Julie Killian (REF),Larry Sharpe and Andrew C. Hollister (LBT),Stephanie A. Miner and Michael J. Volpe (SAM),Blank,Void,Scattering,Total Votes by County
County,,,,,,,,,,,,,,
Albany County,54909,38357,6725,3543,2489,1408,886,835,1867,3007,2807,268,195,"117,296"
Allegany County,3254,9234,833,183,80,106,46,65,963,117,462,3,7,"15,353"


In [82]:
o_gov = official_county_level_election_results_df_gov[[x for x in official_county_level_election_results_df_gov.columns if '(' in x]]
o_gov.columns = [x.split('(')[1][:3] for x in o_gov.columns]
o_gov.head()

,DEM,REP,CON,GRE,WOR,IND,WEP,REF,LBT,SAM
County,,,,,,,,,,
Albany County,54909,38357,6725,3543,2489,1408,886,835,1867,3007
Allegany County,3254,9234,833,183,80,106,46,65,963,117
Broome County,29065,32865,3244,1320,1144,931,379,474,2000,802
Cattaraugus County,6545,13240,1629,307,146,265,78,159,1060,192
Cayuga County,9112,12728,1799,520,232,333,106,179,1023,787


In [83]:
res_gov = piv.groupby('county').sum()[[x for x in piv.columns if x.startswith('Gov')]]
res_gov.head()

,Gov_DEM,Gov_REP,Gov_CON,Gov_GRE,Gov_WOR,Gov_IND,Gov_WEP,Gov_REF,Gov_SAM,Gov_LBT
county,,,,,,,,,,
Albany County,54909,38357,6725,3543,2489,1408,886,835,3007,1867
Allegany County,12488,0,833,183,80,106,46,65,117,963
Bronx County,253712,20401,2407,2828,4198,2143,503,271,593,1074
Broome County,29065,32865,3244,1320,1144,931,379,474,802,2000
Cattaraugus County,6545,13240,1629,307,146,265,78,159,192,1060


In [84]:
res_gov.columns = [x.split('_')[1] for x in res_gov.columns]
res_gov.head()

,DEM,REP,CON,GRE,WOR,IND,WEP,REF,SAM,LBT
county,,,,,,,,,,
Albany County,54909,38357,6725,3543,2489,1408,886,835,3007,1867
Allegany County,12488,0,833,183,80,106,46,65,117,963
Bronx County,253712,20401,2407,2828,4198,2143,503,271,593,1074
Broome County,29065,32865,3244,1320,1144,931,379,474,802,2000
Cattaraugus County,6545,13240,1629,307,146,265,78,159,192,1060


In [85]:
party_lst = list(parties)
party_lst.sort()
party_lst

['CON', 'DEM', 'GRE', 'IND', 'LBT', 'REF', 'REP', 'SAM', 'WEP', 'WOR']

Order columns the same way in both data sets

In [86]:
o_gov = o_gov[party_lst]
o_gov.head(2)

,CON,DEM,GRE,IND,LBT,REF,REP,SAM,WEP,WOR
County,,,,,,,,,,
Albany County,6725,54909,3543,1408,1867,835,38357,3007,886,2489
Allegany County,833,3254,183,106,963,65,9234,117,46,80


In [87]:
res_gov = res_gov[party_lst]
res_gov.head(2)

,CON,DEM,GRE,IND,LBT,REF,REP,SAM,WEP,WOR
county,,,,,,,,,,
Albany County,6725,54909,3543,1408,1867,835,38357,3007,886,2489
Allegany County,833,12488,183,106,963,65,0,117,46,80


### Compare party by party and also total votes cast in each county between Open Elections and official data

#### Total votes cast for the race

In [100]:
percent_diff = res_gov.sum(axis=1)/o_gov.sum(axis=1)
acceptable_percent_diff = .05
lower_bound = 1 - acceptable_percent_diff
upper_bound = 1 + acceptable_percent_diff
print("Accepting ratios between {} and {}".format(lower_bound, upper_bound))
print("The outliers are: ")
outliers = percent_diff[(percent_diff < lower_bound) | (percent_diff > upper_bound)]
outliers

Accepting ratios between 0.95 and 1.05
The outliers are: 


Lewis County    0.928737
dtype: float64

In [101]:
total_votes_diff = res_gov.sum(axis=1) - o_gov.sum(axis=1)
non_zero_votes_diff = total_votes_diff[total_votes_diff.index.isin(outliers.index)]
non_zero_votes_diff

Lewis County   -645.0
dtype: float64

In [102]:
o_gov[o_gov.index.isin(outliers.index)]

,CON,DEM,GRE,IND,LBT,REF,REP,SAM,WEP,WOR
County,,,,,,,,,,
Lewis County,534,1718,102,71,412,74,5945,130,31,34


In [103]:
res_gov[res_gov.index.isin(outliers.index)].sum(axis=1)

county
Lewis County    8406
dtype: int64

#### Save the file

In [105]:
export_df = piv.reset_index()
export_df.head()

,county,precinct,HOR_DEM,HOR_REP,HOR_CON,HOR_GRE,HOR_WOR,HOR_IND,HOR_WEP,HOR_REF,...,Gov_DEM,Gov_REP,Gov_CON,Gov_GRE,Gov_WOR,Gov_IND,Gov_WEP,Gov_REF,Gov_SAM,Gov_LBT
0,Albany County,0001 ALBANY W1 ED1,40,7,0,0,5,0,1,0,...,35,9,1,3,3,0,0,0,2,1
1,Albany County,0002 ALBANY W1 ED2,182,20,0,0,21,0,6,3,...,148,32,11,15,11,1,1,2,9,3
2,Albany County,0003 ALBANY W1 ED3,241,41,0,0,11,0,1,4,...,212,55,11,4,10,1,3,0,5,5
3,Albany County,0004 ALBANY W1 ED4,245,18,0,0,18,0,5,3,...,220,30,5,4,12,4,1,3,7,4
4,Albany County,0005 ALBANY W1 ED5,4,0,0,0,0,0,0,0,...,2,0,0,1,0,0,0,0,0,0


In [106]:
export_df.to_csv('data/precinct_election_results/NY_G18_pivoted.csv', index=False)